# 🚀 Google Colab GPU Runtime Verification & Configuration

This notebook verifies and configures the Google Colab runtime for ML/DL workflows.

**Steps:**
1. GPU Detection & Type Identification
2. CUDA/cuDNN Compatibility Check
3. System Info (Python, RAM)
4. Library Installation & Verification
5. Reproducibility Seeds
6. Environment Summary Report

## 1️⃣ GPU Detection & nvidia-smi

In [10]:
# Check GPU availability using nvidia-smi
print("=" * 60)
print("NVIDIA-SMI Output")
print("=" * 60)
!nvidia-smi

NVIDIA-SMI Output
Wed Jan  7 07:12:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             32W /   70W |     120MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------

In [11]:
# GPU Detection with PyTorch
import torch

print("=" * 60)
print("PyTorch GPU Detection")
print("=" * 60)

cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

if cuda_available:
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPUs: {gpu_count}")
    
    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        gpu_capability = torch.cuda.get_device_capability(i)
        print(f"\nGPU {i}: {gpu_name}")
        print(f"  Compute Capability: {gpu_capability[0]}.{gpu_capability[1]}")
        
        # Memory info
        total_memory = torch.cuda.get_device_properties(i).total_memory / (1024**3)
        print(f"  Total VRAM: {total_memory:.2f} GB")
else:
    print("⚠️ No GPU detected. Please enable GPU runtime in Colab:")
    print("   Runtime → Change runtime type → Hardware accelerator → GPU")

PyTorch GPU Detection
CUDA Available: True
Number of GPUs: 1

GPU 0: Tesla T4
  Compute Capability: 7.5
  Total VRAM: 14.74 GB


## 2️⃣ CUDA & cuDNN Compatibility Check

In [12]:
print("=" * 60)
print("CUDA & cuDNN Verification")
print("=" * 60)

# CUDA Version from PyTorch
print(f"PyTorch CUDA Version: {torch.version.cuda}")

# cuDNN Version
cudnn_version = torch.backends.cudnn.version()
print(f"cuDNN Version: {cudnn_version}")
print(f"cuDNN Enabled: {torch.backends.cudnn.enabled}")

# Check CUDA compilation tools
print("\n--- NVCC Version ---")
!nvcc --version 2>/dev/null || echo "nvcc not found (optional)"

# Verify CUDA is functional
if torch.cuda.is_available():
    try:
        test_tensor = torch.tensor([1.0, 2.0, 3.0]).cuda()
        result = test_tensor * 2
        print(f"\n✅ CUDA Test Passed: {result.cpu().numpy()}")
    except Exception as e:
        print(f"\n❌ CUDA Test Failed: {e}")

CUDA & cuDNN Verification
PyTorch CUDA Version: 12.6
cuDNN Version: 91002
cuDNN Enabled: True

--- NVCC Version ---
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0

✅ CUDA Test Passed: [2. 4. 6.]


## 3️⃣ Python Version & System RAM

In [13]:
import sys
import platform
import psutil
import os

print("=" * 60)
print("System Information")
print("=" * 60)

# Python Version
print(f"Python Version: {sys.version}")
print(f"Platform: {platform.platform()}")

# CPU Information
cpu_count = os.cpu_count()
print(f"\nCPU Cores: {cpu_count}")

# RAM Information
ram_info = psutil.virtual_memory()
total_ram_gb = ram_info.total / (1024**3)
available_ram_gb = ram_info.available / (1024**3)
used_ram_gb = ram_info.used / (1024**3)

print(f"\nTotal RAM: {total_ram_gb:.2f} GB")
print(f"Available RAM: {available_ram_gb:.2f} GB")
print(f"Used RAM: {used_ram_gb:.2f} GB ({ram_info.percent}%)")

System Information
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35

CPU Cores: 2

Total RAM: 12.67 GB
Available RAM: 11.13 GB
Used RAM: 1.22 GB (12.2%)


## 4️⃣ Install & Verify Required Libraries

In [14]:
# Verify all library installations
print("=" * 60)
print("Library Version Verification")
print("=" * 60)

import numpy as np
import pandas as pd
import scipy
import sklearn
import torch
import torchvision
import matplotlib
import seaborn as sns
import tqdm

libraries = {
    "NumPy": np.__version__,
    "Pandas": pd.__version__,
    "SciPy": scipy.__version__,
    "Scikit-learn": sklearn.__version__,
    "PyTorch": torch.__version__,
    "TorchVision": torchvision.__version__,
    "Matplotlib": matplotlib.__version__,
    "Seaborn": sns.__version__,
    "tqdm": tqdm.__version__,
}

for lib, version in libraries.items():
    print(f"{lib:15} : {version}")

# Verify PyTorch CUDA support
print(f"\n{'PyTorch CUDA':15} : {torch.cuda.is_available()}")
print(f"{'PyTorch cuDNN':15} : {torch.backends.cudnn.is_available()}")

Library Version Verification
NumPy           : 2.0.2
Pandas          : 2.2.2
SciPy           : 1.16.3
Scikit-learn    : 1.6.1
PyTorch         : 2.9.0+cu126
TorchVision     : 0.24.0+cu126
Matplotlib      : 3.10.0
Seaborn         : 0.13.2
tqdm            : 4.67.1

PyTorch CUDA    : True
PyTorch cuDNN   : True


## 5️⃣ Set Global Random Seeds for Reproducibility

In [15]:
import random
import numpy as np
import torch

def set_seed(seed: int = 42):
    """
    Set random seeds for reproducibility across all libraries.
    
    Args:
        seed: Random seed value (default: 42)
    """
    # Python random
    random.seed(seed)
    
    # NumPy
    np.random.seed(seed)
    
    # PyTorch
    torch.manual_seed(seed)
    
    # CUDA seeds (if available)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU
        
        # Ensure deterministic behavior (may impact performance)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
    print(f"✅ Random seeds set to {seed} for reproducibility")
    print(f"   - Python random: {seed}")
    print(f"   - NumPy: {seed}")
    print(f"   - PyTorch: {seed}")
    if torch.cuda.is_available():
        print(f"   - CUDA: {seed}")
        print(f"   - cuDNN deterministic: True")

# Set the seed
SEED = 42
set_seed(SEED)

✅ Random seeds set to 42 for reproducibility
   - Python random: 42
   - NumPy: 42
   - PyTorch: 42
   - CUDA: 42
   - cuDNN deterministic: True


## 6️⃣ Environment Summary Report

In [16]:
import sys
import os
import psutil
import torch

def generate_environment_report():
    """
    Generate a clean environment summary report.
    """
    print("\n" + "=" * 60)
    print("🖥️  ENVIRONMENT SUMMARY REPORT")
    print("=" * 60)
    
    # GPU Information
    print("\n📊 GPU Configuration")
    print("-" * 40)
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        cuda_version = torch.version.cuda
        
        # Get current GPU memory usage
        allocated = torch.cuda.memory_allocated(0) / (1024**3)
        reserved = torch.cuda.memory_reserved(0) / (1024**3)
        free_vram = gpu_memory - reserved
        
        print(f"  GPU Name      : {gpu_name}")
        print(f"  CUDA Version  : {cuda_version}")
        print(f"  Total VRAM    : {gpu_memory:.2f} GB")
        print(f"  Free VRAM     : {free_vram:.2f} GB")
        print(f"  Allocated     : {allocated:.4f} GB")
    else:
        print("  ⚠️ No GPU available")
    
    # PyTorch Information
    print("\n🔥 PyTorch Configuration")
    print("-" * 40)
    print(f"  PyTorch Version : {torch.__version__}")
    print(f"  CUDA Available  : {torch.cuda.is_available()}")
    print(f"  cuDNN Available : {torch.backends.cudnn.is_available()}")
    if torch.backends.cudnn.is_available():
        print(f"  cuDNN Version   : {torch.backends.cudnn.version()}")
    
    # CPU & RAM Information
    print("\n💻 System Configuration")
    print("-" * 40)
    print(f"  Python Version  : {sys.version.split()[0]}")
    print(f"  CPU Cores       : {os.cpu_count()}")
    
    ram = psutil.virtual_memory()
    print(f"  Total RAM       : {ram.total / (1024**3):.2f} GB")
    print(f"  Available RAM   : {ram.available / (1024**3):.2f} GB")
    print(f"  RAM Usage       : {ram.percent}%")
    
    # Final CUDA Check
    print("\n" + "=" * 60)
    print("🔍 FINAL CUDA AVAILABILITY CHECK")
    print("=" * 60)
    cuda_status = torch.cuda.is_available()
    if cuda_status:
        print("\n  ✅ CUDA is AVAILABLE and ready for use!")
        print(f"     Device: {torch.cuda.get_device_name(0)}")
    else:
        print("\n  ❌ CUDA is NOT available.")
        print("     Please enable GPU runtime in Colab.")
    
    print("\n" + "=" * 60)
    return cuda_status

# Generate the report
cuda_ready = generate_environment_report()


🖥️  ENVIRONMENT SUMMARY REPORT

📊 GPU Configuration
----------------------------------------
  GPU Name      : Tesla T4
  CUDA Version  : 12.6
  Total VRAM    : 14.74 GB
  Free VRAM     : 14.74 GB
  Allocated     : 0.0000 GB

🔥 PyTorch Configuration
----------------------------------------
  PyTorch Version : 2.9.0+cu126
  CUDA Available  : True
  cuDNN Available : True
  cuDNN Version   : 91002

💻 System Configuration
----------------------------------------
  Python Version  : 3.12.12
  CPU Cores       : 2
  Total RAM       : 12.67 GB
  Available RAM   : 11.13 GB
  RAM Usage       : 12.2%

🔍 FINAL CUDA AVAILABILITY CHECK

  ✅ CUDA is AVAILABLE and ready for use!
     Device: Tesla T4



---

## ✅ Environment Ready!

If all checks passed, your Google Colab GPU runtime is configured and ready for ML/DL workflows.

**Next Steps:**
- Load your dataset
- Define your model architecture
- Start training!

---